## Deliverable 3. Create a Travel Itinerary Map.

In [1]:
# Dependencies
%matplotlib inline

import matplotlib.pyplot as plt
import json
import requests
import os
import pandas as pd
import numpy as np
import gmaps
import requests

from scipy.stats import linregress

from citipy import citipy
from datetime import datetime
import time

# Import API key
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv("OpenWeather_API_KEY")
g_key = os.getenv("g_key")

# Configure gmaps
gmaps.configure(api_key=g_key)


In [2]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.
vacation_df = pd.read_csv("../vacation_Search/WeatherPy_vacation.csv")
vacation_df = vacation_df.set_index("City_ID")
vacation_df.head()

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
City_ID,,,,,,,
19,Surt,LY,78.62,clear sky,31.2089,16.5887,الهادي ابوشعاله سرت
27,Carnarvon,AU,76.87,broken clouds,-24.8667,113.6333,Hospitality Carnarvon
55,Alofi,NU,75.09,clear sky,-19.0595,-169.9187,Taloa Heights
65,Mutis,CO,77.97,moderate rain,6.2262,-77.4044,Paraiso Escondido
72,Ambilobe,MG,78.75,clear sky,-13.2000,49.0500,Hôtel & Restaurant DIANA Ambilobe


In [3]:
# 2. Using the template add the city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Description</dt><dd>{Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 3a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

# 3b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = vacation_df[["Lat", "Lng"]]


In [4]:
# 4a. Add a marker layer for each city to the map.
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)


# 4b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# Hint: The starting and ending city should be the same city.

# Norfolk

vacation_start = vacation_df.loc[(vacation_df['City'] == 'Norfolk')]


# Norfolk
vacation_end = vacation_df.loc[(vacation_df['City'] == 'Norfolk')]

# Palmer
vacation_stop1 = vacation_df.loc[(vacation_df['City'] == 'Palmer')]

# Grafton
vacation_stop2 = vacation_df.loc[(vacation_df['City'] == 'Grafton')]

#Roma
vacation_stop3 = vacation_df.loc[(vacation_df['City'] == 'Roma')]


In [6]:
vacation_stop2

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
City_ID,,,,,,,
326,Grafton,US,75.45,clear sky,42.207,-71.6856,The Grafton Inn


In [7]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
start = (vacation_start["Lat"].to_numpy()[0],vacation_start["Lng"].to_numpy()[0])
end = (vacation_end["Lat"].to_numpy()[0],vacation_end["Lng"].to_numpy()[0])
stop1 = (vacation_stop1["Lat"].to_numpy()[0],vacation_stop1["Lng"].to_numpy()[0])
stop2 = (vacation_stop2["Lat"].to_numpy()[0],vacation_stop2["Lng"].to_numpy()[0])
stop3 = (vacation_stop3["Lat"].to_numpy()[0],vacation_stop3["Lng"].to_numpy()[0])

In [8]:
vacation_start

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
City_ID,,,,,,,
427,Norfolk,US,78.08,clear sky,42.1668,-71.2495,Residence Inn by Marriott Boston Norwood/Canton


In [9]:
start

(42.1668, -71.2495)

In [10]:
# 7. Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".

fig = gmaps.figure()
directions_layer = gmaps.directions_layer(start, end, waypoints=[stop1, stop2, stop3], travel_mode='DRIVING')
fig.add_layer(directions_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [11]:
# 8. To create a marker layer map between the four cities.
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat([vacation_start,vacation_stop1,vacation_stop2, vacation_stop3, vacation_end],ignore_index=True)
itinerary_df

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Norfolk,US,78.08,clear sky,42.1668,-71.2495,Residence Inn by Marriott Boston Norwood/Canton
1,Palmer,US,75.61,clear sky,42.1584,-72.3287,Wedgewood Motel
2,Grafton,US,75.45,clear sky,42.2070,-71.6856,The Grafton Inn
3,Roma,US,75.09,overcast clouds,43.2128,-75.4557,Wingate By Wyndham Rome
4,Norfolk,US,78.08,clear sky,42.1668,-71.2495,Residence Inn by Marriott Boston Norwood/Canton


In [12]:
# 9 Using the template add city name, the country code, the weather description and maximum temperature for the city. 
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Description</dt><dd>{Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# 10a Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(start), zoom_level=6)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)


## BUG- Direction layer doesn't seem to run alongside marker_layer
## fig.add_layer(directions_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

# 4a. Add a marker layer for each city to the map.
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig

Figure(layout=FigureLayout(height='420px'))